In [8]:
#Data preprocessing
import pandas as pd

from numpy import isnan
from numpy import nan
from pandas import read_csv

dir = 'CNN_TS_data'
fname = 'household_power_consumption.txt'
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col='datetime')

# mark all missing values
dataset.replace('?', nan, inplace=True)
# make dataset numeric
dataset = dataset.astype('float32')

# fill missing values with a value at the same time one day ago
def fill_missing(values):
	one_day = 60 * 24
	for row in range(values.shape[0]):
		for col in range(values.shape[1]):
			if isnan(values[row, col]):
				values[row, col] = values[row - one_day, col]

# fill missing
fill_missing(dataset.values)

# save updated dataset
dataset.to_csv('household_power_consumption.csv')


In [9]:
# resample minute data to total for half hour 
# load the new file
dataset = read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

# resample data to half hour data
halfhour_groups = dataset.resample('30min')
halfhour_data = halfhour_groups.sum()

# save
halfhour_data.to_csv('household_power_consumption_halfhour.csv')


In [10]:
#Seperate date and time from index
halfhour_data['Date'] = [d.date() for d in halfhour_data.index]
halfhour_data['Time'] = [d.time() for d in halfhour_data.index]

#Seperate column for day, month and year
halfhour_data['day'] = halfhour_data.index.day
halfhour_data['month'] = halfhour_data.index.month
halfhour_data['year'] = halfhour_data.index.year

#Seperate hour, minute and seconds from time
halfhour_data[['h','m','s']] = halfhour_data['Time'].astype(str).str.split(':', expand=True).astype(int)

#Assign day of week
halfhour_data['day_of_week'] = halfhour_data.index.day_name()

In [7]:
weekend_flag = halfhour_data.weekend_flag


AttributeError: 'DataFrame' object has no attribute 'weekend_flag'

In [6]:
#Save to csv file
halfhour_data.to_csv('household_power_consumption_halfhour.csv')